In [1]:
!pip install onnxruntime==1.16.0
!pip install opencv-python==4.8.1.78


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import onnxruntime as rt

In [3]:
#import remote_infer as ri
import remote_infer_changed_preprocessing as ri

In [4]:
image_path = 'images/carImage3.jpg' # You can replace this with an image you upload

In [5]:
preprocessed, scale, original_image = ri.preprocess(image_path)

# Serving

In [6]:
infer_url = os.environ["INFER_ENDPOINT"]

response = ri.send_request(preprocessed, infer_url)

In [7]:
response_transposed = cv2.transpose(response[0].squeeze())
print(response_transposed.shape)
print(response_transposed)

(8400, 6)
[[7.25182250e+00 1.59619200e+01 1.59490360e+01 3.18928570e+01
  2.77792900e-06 8.99698350e-06]
 [2.54508700e+01 1.32270820e+01 5.91368030e+01 2.61576290e+01
  1.19057480e-06 9.94917500e-06]
 [1.81783560e+01 1.16864980e+01 4.00377120e+01 2.24915900e+01
  3.50209300e-07 4.00328800e-06]
 ...
 [5.63258540e+02 6.07380500e+02 1.35587590e+02 6.58579100e+01
  2.88683740e-06 3.79249760e-06]
 [5.84164300e+02 5.98486000e+02 1.04063416e+02 8.50695800e+01
  2.83189700e-06 3.47373320e-06]
 [6.00892500e+02 5.86694700e+02 9.20753200e+01 1.19198730e+02
  4.63818740e-06 7.07730670e-06]]


# Session

In [8]:
sess = rt.InferenceSession("accident_detect.onnx", providers=rt.get_available_providers())
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

In [9]:
sess_prediction = sess.run([output_name], {input_name: preprocessed.tolist()})

In [10]:
sess_prediction_transposed = cv2.transpose(sess_prediction[0].squeeze())
print(sess_prediction_transposed.shape)
print(sess_prediction_transposed)

(8400, 6)
[[7.2518239e+00 1.5961920e+01 1.5949042e+01 3.1892857e+01 2.7716160e-06
  8.9406967e-06]
 [2.5450851e+01 1.3227080e+01 5.9136772e+01 2.6157629e+01 1.1920929e-06
  9.8943710e-06]
 [1.8178358e+01 1.1686497e+01 4.0037716e+01 2.2491585e+01 3.5762787e-07
  3.9339066e-06]
 ...
 [5.6325854e+02 6.0738049e+02 1.3558755e+02 6.5857849e+01 2.8610229e-06
  3.8146973e-06]
 [5.8416425e+02 5.9848602e+02 1.0406348e+02 8.5069580e+01 2.8312206e-06
  3.4272671e-06]
 [6.0089258e+02 5.8669470e+02 9.2075378e+01 1.1919873e+02 4.6193600e-06
  7.0631504e-06]]


# Compare them

In [11]:
def compare_2d_arrays(arr1, arr2):
    # Ensure arr1 and arr2 have the same shape
    if arr1.shape != arr2.shape:
        raise ValueError("Arrays must have the same shape")
    
    differences = []  # List to store differences for each row
    
    # Iterate over each row
    for row_idx in range(arr1.shape[0]):
        row_differences = []  # Differences for the current row
        for col_idx in range(arr1.shape[1]):
            item1 = arr1[row_idx, col_idx]
            item2 = arr2[row_idx, col_idx]
            diff = abs(item1 - item2)  # Calculate the absolute difference
            row_differences.append((col_idx, item1, item2, diff))
        
        differences.append(row_differences)
    
    return differences

# Comparing the arrays
differences = compare_2d_arrays(response_transposed, sess_prediction_transposed)

# Optionally, print the results for the first few rows to check
for row_idx, row_diff in enumerate(differences[:5]):  # Just as an example, print differences for the first 5 rows
    print(f"Row {row_idx}:")
    for diff in row_diff:
        print(f"  Col {diff[0]}: {diff[1]} (rest_prediction) vs {diff[2]} (sess_prediction) - Difference:  {diff[3]}")


Row 0:
  Col 0: 7.2518225 (rest_prediction) vs 7.251823902130127 (sess_prediction) - Difference:  1.4021301266708974e-06
  Col 1: 15.96192 (rest_prediction) vs 15.961919784545898 (sess_prediction) - Difference:  2.154541007826083e-07
  Col 2: 15.949036 (rest_prediction) vs 15.949042320251465 (sess_prediction) - Difference:  6.320251465297133e-06
  Col 3: 31.892857 (rest_prediction) vs 31.89285659790039 (sess_prediction) - Difference:  4.0209960872061856e-07
  Col 4: 2.777929e-06 (rest_prediction) vs 2.771615982055664e-06 (sess_prediction) - Difference:  6.313017944335798e-09
  Col 5: 8.9969835e-06 (rest_prediction) vs 8.940696716308594e-06 (sess_prediction) - Difference:  5.628678369140654e-08
Row 1:
  Col 0: 25.45087 (rest_prediction) vs 25.450851440429688 (sess_prediction) - Difference:  1.8559570310827667e-05
  Col 1: 13.227082 (rest_prediction) vs 13.227080345153809 (sess_prediction) - Difference:  1.6548461907461842e-06
  Col 2: 59.136803 (rest_prediction) vs 59.13677215576172 (se